<a href="https://colab.research.google.com/github/Tissyjohnson/DSA_Class_Activities/blob/main/ETL_Casestudy_Tissy_Johnson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [3]:
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
df = pd.read_csv(url)
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-09-21T07:34:48.073Z,-13.616900,122.376400,10.00,4.90,mb,33.0,53.0,4.26100,0.72,...,2025-09-21T07:51:54.040Z,northwest of Australia,earthquake,7.86,1.890,0.112,25.0,reviewed,us,us
1,2025-09-21T07:29:37.335Z,61.388600,-146.597800,27.20,1.40,ml,NaN,NaN,NaN,1.04,...,2025-09-21T07:31:55.178Z,"31 km NNW of Valdez, Alaska",earthquake,NaN,0.300,NaN,NaN,automatic,ak,ak
2,2025-09-21T07:14:56.510Z,33.339833,-116.287833,11.74,0.97,ml,56.0,36.0,0.02893,0.21,...,2025-09-21T07:18:21.644Z,"12 km NE of Borrego Springs, CA",earthquake,0.20,0.300,0.148,27.0,automatic,ci,ci
3,2025-09-21T07:14:21.263Z,53.168000,161.211900,10.00,4.60,mb,41.0,113.0,1.55200,0.86,...,2025-09-21T07:35:16.040Z,"173 km E of Petropavlovsk-Kamchatsky, Russia",earthquake,6.66,1.835,0.073,56.0,reviewed,us,us
4,2025-09-21T07:01:58.208Z,67.361700,-156.744900,1.40,2.50,ml,NaN,NaN,NaN,0.51,...,2025-09-21T07:06:42.656Z,"51 km N of Kobuk, Alaska",earthquake,NaN,0.600,NaN,NaN,automatic,ak,ak


In [4]:
num_records = len(df)
unique_places = df['place'].nunique()

print("Total Records:", num_records)
print("Unique Locations:", unique_places)


Total Records: 9929
Unique Locations: 5263


In [6]:
print(df.head())



                       time   latitude   longitude  depth   mag magType   nst  \
0  2025-09-21T07:34:48.073Z -13.616900  122.376400  10.00  4.90      mb  33.0   
1  2025-09-21T07:29:37.335Z  61.388600 -146.597800  27.20  1.40      ml   NaN   
2  2025-09-21T07:14:56.510Z  33.339833 -116.287833  11.74  0.97      ml  56.0   
3  2025-09-21T07:14:21.263Z  53.168000  161.211900  10.00  4.60      mb  41.0   
4  2025-09-21T07:01:58.208Z  67.361700 -156.744900   1.40  2.50      ml   NaN   

     gap     dmin   rms  ...                   updated  \
0   53.0  4.26100  0.72  ...  2025-09-21T07:51:54.040Z   
1    NaN      NaN  1.04  ...  2025-09-21T07:31:55.178Z   
2   36.0  0.02893  0.21  ...  2025-09-21T07:18:21.644Z   
3  113.0  1.55200  0.86  ...  2025-09-21T07:35:16.040Z   
4    NaN      NaN  0.51  ...  2025-09-21T07:06:42.656Z   

                                          place        type horizontalError  \
0                        northwest of Australia  earthquake            7.86   
1     

In [7]:
df['time'] = pd.to_datetime(df['time'])


In [9]:
print(df.dtypes)

time               datetime64[ns, UTC]
latitude                       float64
longitude                      float64
depth                          float64
mag                            float64
magType                         object
nst                            float64
gap                            float64
dmin                           float64
rms                            float64
net                             object
id                              object
updated                         object
place                           object
type                            object
horizontalError                float64
depthError                     float64
magError                       float64
magNst                         float64
status                          object
locationSource                  object
magSource                       object
dtype: object


In [11]:
df = df.dropna(subset=['latitude', 'longitude', 'mag'])

In [12]:
df = df[df['mag'] >= 4.0]

In [13]:
df['day_of_week'] = df['time'].dt.day_name()

In [16]:
def get_severity(mag):
    if mag < 4.0:
        return "Low"
    elif mag < 6.0:
        return "Moderate"
    else:
        return "High"

df['severity_level'] = df['mag'].apply(get_severity)


In [20]:
earthquakes_per_place = df['place'].value_counts().reset_index()
earthquakes_per_place.columns = ['place', 'earthquake_count']


In [21]:
summary = df.groupby(df['time'].dt.date).agg(
    avg_magnitude=('mag', 'mean'),
    max_depth=('depth', 'max')
).reset_index()


In [22]:
df.to_csv("cleaned_earthquakes.csv", index=False)
summary.to_csv("earthquake_summary.csv", index=False)


In [23]:
import sqlite3

conn = sqlite3.connect("earthquakes.db")
df.to_sql("earthquakes", conn, if_exists="replace", index=False)
summary.to_sql("earthquake_summary", conn, if_exists="replace", index=False)
conn.close()


**13. Flowchart of ETL Logic**

Flow:
Extract CSV → Load DataFrame → Clean Data (drop NA, filter magnitude) → Transform (datetime, severity, day_of_week) → Aggregate (per place & per day) → Save (CSV + SQLite)

14: Short explanation of the process

We performed an ETL process on USGS earthquake data: loaded the CSV, cleaned missing values, filtered for significant earthquakes (mag ≥ 4), added derived columns (day_of_week and severity_level), aggregated daily statistics, and saved both cleaned and summary datasets. We also stored the data in SQLite for further queries.

15: One insight from the data

Most significant earthquakes (magnitude ≥ 4) in the past month occurred in specific regions like Alaska and Indonesia, with the majority being of Moderate severity. This shows regional concentration of seismic activity.